In [1]:
import numpy as np
import pandas as pd
import re
from bs4 import BeautifulSoup
from nltk.tokenize import WordPunctTokenizer
from sklearn.metrics import classification_report

import matplotlib.pyplot as plt
import seaborn as sns
% matplotlib inline
import time
import requests

In [11]:
tweets = pd.read_pickle('./Data/Raw/tweetsSpamClf.pkl')

In [16]:
tweets.columns

Index(['ID', 'datetime', 'is_reply', 'is_retweet', 'nbr_favorite', 'nbr_reply',
       'nbr_retweet', 'text', 'url', 'user_id', 'usernameTweet', 'has_media',
       'medias', 'spam', 'P_spam'],
      dtype='object')

In [14]:
for t in tweets[~tweets["spam"]]['text'][0:50]:
    print(t)
    print()
    print()

New pos (Could  Ripple   XRP  Relly Rech $1. I 39 Dys?) hs bee published o  -  hps: msscrypocurrecy.com?p=2664   pic.wier.com6Dlog2TNQE


LogShor Bicoi swigs wih up o 1 Leverge  BiMEX!

1% Fee Discou usig he lik below  

→  hp: bime.comregiserm9MNm   

 $ LSK   $ BCH   $ DGB   $ AION   $ EMC   $ RDD   $ SALT   $ OMG   $ ARK   $ EOS   $ MAID   $ WTC   $ QTUM   $ XRP   $ LTC   $ ELF   $ ZEC   $ BAT pic.wier.comIFj1YJkO


Did you kow h you c buy, sell, sore, cover d rsfer @Ripple  #XRP  o  hp: Coidirec.com       i he UK, EU couries, Ausrli, Key, Nigeri d Souh Afric. You c use fi currecy or cover oher cois d echge hem for #Ripple  .


1  Ripple  = .6589 USD.  Ripple  hs chged by -.26 USD i 3 mis. Live price:  hps: is.gdhI9OcA    #ripple  #rp  #crypocurrecy


How Is #Ripple  Differe From All Oher #Crypocurrecies ? A Ulime Guide  hps:www. reddi.comrRipplecomme s8d58hrhow_is_ripple_differe_from_ll_oher   … 
#ripple  #ro   $ r   $ rp   $ eh   $ bc   $ mr   $ lc  #liecoi   $ ec   $ eos   $ eo  

In [15]:
tweets['rounded_dateTime'].describe()

KeyError: 'rounded_dateTime'

In [39]:
tweets.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 83317 entries, 0 to 85534
Data columns (total 19 columns):
ID                  83317 non-null float64
datetime            83317 non-null datetime64[ns]
is_reply            83317 non-null float64
is_retweet          83317 non-null float64
nbr_favorite        83317 non-null float64
nbr_reply           83317 non-null float64
nbr_retweet         83317 non-null float64
text                83317 non-null object
url                 83317 non-null object
user_id             83317 non-null object
usernameTweet       83317 non-null object
has_media           28275 non-null float64
medias              28275 non-null object
spam                83317 non-null bool
P_spam              83317 non-null float64
rounded_dateTime    83317 non-null datetime64[ns]
timeId              83317 non-null float64
time                83317 non-null datetime64[ns]
pDelta              83317 non-null float64
dtypes: bool(1), datetime64[ns](3), float64(10), object(5)
me

In [81]:
# URLS
tweets['text'] = tweets['text'].apply(lambda t : re.sub('https?://[A-Za-z0-9./]+',' ',t))

#Mentions
tweets['text'] = tweets['text'].apply(lambda t : re.sub(r'@[A-Za-z0-9]+',' ',t))

#Hashtags
tweets['text'] = tweets['text'].apply(lambda t : re.sub("#", " ", t))

#lowercase
tweets['text'] = tweets['text'].str.lower()

In [82]:
tweets['text'].head()

0    lost me on that one, nick  \nthink it's the te...
1    not only that, but freedom of speech is only p...
2    a fantastic video  from  the archives - coinsc...
3    every morning i read through the posts  from  ...
4    "ask them why they idle there\nwhile we suffer...
Name: text, dtype: object

In [83]:
tok = WordPunctTokenizer()
pat1 = r'@[A-Za-z0-9]+'
pat2 = r'https?://[A-Za-z0-9./]+'
combined_pat = r'|'.join((pat1, pat2))
def tweet_cleaner(text):
    soup = BeautifulSoup(text, 'lxml')
    souped = soup.get_text()
    stripped = re.sub(combined_pat, '', souped)
    try:
        clean = stripped.decode("utf-8-sig").replace(u"\ufffd", "?")
    except:
        clean = stripped
    letters_only = re.sub("[^a-zA-Z]", " ", clean)
    lower_case = letters_only.lower()
    
    # During the letters_only process two lines above, it has created unnecessay white spaces,
    # I will tokenize and join together to remove unneccessary white spaces
    
    words = tok.tokenize(lower_case)
    return (" ".join(words)).strip()

In [84]:
tweets['text'] = tweets['text'].apply(lambda t : tweet_cleaner(t))

In [85]:
from textblob import TextBlob
 
class TwitterClient(object):
    '''
    Generic Twitter Class for sentiment analysis.
    '''
    def __init__(self):
        '''
        Class constructor or initialization method.
        '''
 
    def clean_tweet(self, tweet):
        '''
        Utility function to clean tweet text by removing links, special characters
        using simple regex statements.
        '''
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])"\
                                    "|(\w+:\/\/\S+)", " ", tweet).split())
 
    def get_tweet_sentiment(self, tweet):
        '''
        Utility function to classify sentiment of passed tweet
        using textblob's sentiment method
        '''
        # create TextBlob object of passed tweet text
        analysis = TextBlob(tweet)
        # set sentiment
        if analysis.sentiment.polarity > 0:
            return 'positive'
        elif analysis.sentiment.polarity == 0:
            return 'neutral'
        else:
            return 'negative'

In [86]:
tc = TwitterClient()

In [87]:
cleaned2 = pd.DataFrame(tweets)

In [88]:
cleaned2['tc_sentiment'] = cleaned2['text'].apply(lambda t : tc.get_tweet_sentiment(t))

In [89]:
cleaned2['text'].head(10)

0    lost me on that one nick think it s the term c...
1    not only that but freedom of speech is only pr...
2    a fantastic video from the archives coinscrum ...
3    every morning i read through the posts from my...
4    ask them why they idle there while we suffer a...
5    first coin group newsletter going out later to...
6    at least we d still have trees to swing from a...
7    we may have been born on rd base but we worked...
8    transactions are unfairly cheap right now segw...
9    deeponion secure your files from the thieves b...
Name: text, dtype: object

In [90]:
preds_df = cleaned2[['pDelta', 'tc_sentiment']]

KeyError: "['pDelta'] not in index"

In [91]:
preds_df.describe()

,pDelta
count,83317.000000
mean,0.001582
std,0.014360
min,-0.059803
25%,-0.005616
50%,0.000709
75%,0.007564
max,0.059349


In [92]:
sentimentClasses = [10, -10, 0, -5, 5, -5 ]

wordLists = [
            "http://www3.nd.edu/~mcdonald/Data/Finance_Word_Lists/LoughranMcDonald_Positive.csv",
          "http://www3.nd.edu/~mcdonald/Data/Finance_Word_Lists/LoughranMcDonald_Negative.csv",
          "http://www3.nd.edu/~mcdonald/Data/Finance_Word_Lists/LoughranMcDonald_Uncertainty.csv",
          "http://www3.nd.edu/~mcdonald/Data/Finance_Word_Lists/LoughranMcDonald_Litigious.csv",
          "http://www3.nd.edu/~mcdonald/Data/Finance_Word_Lists/LoughranMcDonald_ModalStrong.csv",
          "http://www3.nd.edu/~mcdonald/Data/Finance_Word_Lists/LoughranMcDonald_ModalWeak.csv"]

wordDict = {}

for i in range(len(sentimentClasses)):
    url = wordLists[i]
    response = requests.get(url)
    words = response.text.replace('\r\n', ' ').replace("'", "").split(' ')[0:-1]
    for w in words: 
        wordDict[w.lower()] = sentimentClasses[i]

In [93]:
def computeSent(text):
    sent = 0
    for w in text.split(' '):
        if w in wordDict.keys():
            sent = sent + wordDict[w]
            
    return sent

In [94]:
tweets['sentiment'] = tweets['text'].apply(lambda t : computeSent(t))

In [95]:
tweets['sentiment'].value_counts()

 0     8293
-10    2323
 10    1823
-5     1300
 5     1280
-20     625
-15     450
 15     361
 20     307
-30     191
-25     162
 25      63
 30      57
-35      42
-40      39
 40      18
 35      16
-45      12
-50      11
-60       5
-70       3
 45       3
 50       2
-55       1
 55       1
Name: sentiment, dtype: int64

In [68]:
for t in tweets['text'][0:25]:
    print(t)
    print()
    print()

new pos could ripple xrp relly rech i dys hs bee published o hps msscrypocurrecy com p pic wier com dlog tnqe


logshor bicoi swigs wih up o leverge bimex fee discou usig he lik below hp bime comregiserm mnm lsk bch dgb aion emc rdd salt omg ark eos maid wtc qtum xrp ltc elf zec bat pic wier comifj yjko


did you kow h you c buy sell sore cover d rsfer xrp o hp coidirec com i he uk eu couries ausrli key nigeri d souh afric you c use fi currecy or cover oher cois d echge hem for ripple


ripple usd ripple hs chged by usd i mis live price hps is gdhi oca ripple rp crypocurrecy


how is ripple differe from all oher crypocurrecies a ulime guide hps www reddi comrripplecomme s d hrhow is ripple differe from ll oher ripple ro r rp eh bc mr lc liecoi ec eos eo lm d crdo em io lsk ic crypoews redbu


drikig revive by brod ripple hp up beersc


his m fighig for his durg h s o eve doig is job i do see oe ripple hps wier comjplussus


immese pid group ow free for week hps goo glpws qf cge heat sl

In [59]:
def deltaConvert(d):
    if d > 0.005:
        return 'positive'
    elif d < -0.005:
        return 'negative'
    else:
        return 'neutral'

In [60]:
preds_df['actual'] = preds_df['pDelta'].apply(lambda p : deltaConvert(p))

/Users/Braden/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [61]:
print(classification_report(preds_df['actual'], preds_df['tc_sentiment']))

             precision    recall  f1-score   support

   negative       0.24      0.07      0.10      6616
    neutral       0.42      0.52      0.47     10619
   positive       0.33      0.42      0.37      8200

avg / total       0.35      0.37      0.34     25435



In [4]:
from monkeylearn import MonkeyLearn

ml = MonkeyLearn('5214f791698d7ea9b907d922cd6def91f05e1d39')

In [8]:
textList = tweets['text'][0:1000]
textList= textList.tolist()

In [9]:
module_id = 'cl_Jx8qzYJh'
res = ml.classifiers.classify(module_id, textList, sandbox=False)

MonkeyLearnException: Error: "Request Throttled. You exceeded your rate limits for your current plan. In order to raise this limit you need to configure the extra queries charging or upgrade your plan."

In [10]:
print(res.response)

NameError: name 'res' is not defined